# **Topics in Data Science**
### ***Final Project***

In [1]:
import pandas as pd  
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import Audio
import librosa

from sklearn import preprocessing

In [2]:
train_path_real = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/training/real/'
train_path_fake = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/training/fake/'

test_path_real = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/testing/real/'
test_path_fake = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/testing/fake/'

validation_path_real = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/validation/real/'
validation_path_fake = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/validation/fake/'

In [3]:
import os
file_names = os.listdir(train_path_real)
train_real_names = [file for file in file_names if file.endswith(('.wav'))]

file_names = os.listdir(train_path_fake)
train_fake_names = [file for file in file_names if file.endswith(('.wav'))]

file_names = os.listdir(test_path_real)
test_real_names = [file for file in file_names if file.endswith(('.wav'))]

file_names = os.listdir(test_path_fake)
test_fake_names = [file for file in file_names if file.endswith(('.wav'))]

file_names = os.listdir(validation_path_real)
valid_real_names = [file for file in file_names if file.endswith(('.wav'))]

file_names = os.listdir(validation_path_fake)
valid_fake_names = [file for file in file_names if file.endswith(('.wav'))]

In [4]:
from scipy import stats

def get_stats(x):
    if np.any(x) == False:
        return np.nan,np.nan,np.nan,np.nan,np.nan, np.nan, np.nan
    else:
        return x.mean(),x.std(),x.var(),x.min(), x.max(), stats.skew(x),stats.kurtosis(x) 
        #for variance, np.var uses ddof=0 as default, whereas scipy.stats use ddof=1 
        
def get_features(list_file_names, list_file_names_path):
    name_list = []
    f0_list = []
    zcr_list = []
    mfcc_list = []
    for filename in list_file_names:
        x,sr = librosa.load(list_file_names_path+filename, sr=None)
       
        f0, voiced_flag, voiced_probs = librosa.pyin(y=x, sr=sr, fmin=25, fmax = 150, fill_na = None)
        f0_stats = get_stats(f0[voiced_flag])
        f0_list.append(f0_stats)
        
        zcr = librosa.feature.zero_crossing_rate(y=x)
        zcr_stats = get_stats(zcr[0,:])
        zcr_list.append(zcr_stats)
        
        mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=25, dct_type=2)
        mfcc_mean = mfcc.mean(axis=1)
        mfcc_list.append(mfcc_mean)
        
        name_list.append(filename)
    
    stats_name = ['mean','std','var','min','max','skew','kurtosis']
    f0_feature = pd.DataFrame(np.stack(f0_list), columns=["f0_" + sn for sn in stats_name])
    zcr_feature = pd.DataFrame(np.stack(zcr_list), columns=["zcr_" + sn for sn in stats_name])
    mfcc_array = np.stack(mfcc_list,axis=0)
    mfcc_feature = pd.DataFrame(mfcc_array, columns=["mfcc_"+str(n).rjust(2, "0") for n in range(mfcc_array.shape[1])])
    
    voice_feature_new = pd.concat([pd.DataFrame(name_list), f0_feature, zcr_feature, mfcc_feature], axis=1)
    
    return voice_feature_new
        
    

In [5]:
train_fake = get_features(train_fake_names, train_path_fake)
train_fake['label'] = 0
train_fake

/var/folders/99/6591xgpn4sq1ypfgnrzy_7n80000gn/T/ipykernel_42598/3313768297.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return x.mean(),x.std(),x.var(),x.min(), x.max(), stats.skew(x),stats.kurtosis(x)


,0,f0_mean,f0_std,f0_var,f0_min,f0_max,f0_skew,f0_kurtosis,zcr_mean,zcr_std,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,label
0,file24569.mp3.wav_16k.wav_norm.wav_mono.wav_si...,113.146319,16.642187,276.962402,85.559503,143.064593,0.084161,-1.177386,0.164900,0.162481,...,-20.343092,-2.797815,-12.452229,-5.840678,-10.560199,0.486194,-7.003640,7.888074,-7.005135,0
1,file1886.mp3.wav_16k.wav_norm.wav_mono.wav_sil...,116.304929,20.466395,418.873322,89.605964,149.830708,0.278453,-1.560141,0.149197,0.125664,...,-10.500713,-4.870155,-14.144915,-8.112008,-5.849154,-8.864585,-3.883942,-4.553544,-5.014913,0
2,file5633.mp3.wav_16k.wav_norm.wav_mono.wav_sil...,101.062877,18.909091,357.553741,76.666417,139.796993,0.457587,-1.057336,0.101966,0.061466,...,-12.358951,2.754946,-2.687284,0.354355,-7.595921,2.471754,-6.340443,4.280274,-1.238021,0
3,file27375.wav_16k.wav_norm.wav_mono.wav_silenc...,111.522149,16.696594,278.776240,86.055144,147.256706,0.099230,-1.191811,0.183400,0.174183,...,-5.718417,-9.363832,-5.133481,-6.674107,-8.906966,-1.871459,-4.009862,8.324374,7.092777,0
4,file17442.mp3.wav_16k.wav_norm.wav_mono.wav_si...,99.681495,16.756603,280.783737,75.785828,147.256706,1.289548,1.524926,0.164946,0.137211,...,-12.736900,4.043910,-4.198050,3.157935,-5.054732,7.856312,-9.573131,5.844799,-4.269224,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6973,file15286.mp3.wav_16k.wav_norm.wav_mono.wav_si...,123.270983,18.369711,337.446295,85.066716,148.109755,-0.842104,-0.631202,0.114986,0.092294,...,-1.580060,-15.854191,-9.602470,-0.201405,-1.784154,10.320373,7.756118,18.316584,11.194551,0
6974,file23658.mp3.wav_16k.wav_norm.wav_mono.wav_si...,106.408143,19.172657,367.590785,77.557238,149.830708,0.574488,-0.496523,0.126023,0.131931,...,-20.477757,-2.748439,-14.629633,-5.567091,-9.753058,-2.981671,-7.705575,4.244843,-9.142785,0
6975,file22416.mp3.wav_16k.wav_norm.wav_mono.wav_si...,101.458720,12.242650,149.882478,80.292288,128.194408,0.489334,-0.610149,0.115320,0.090018,...,-14.148066,3.685224,-8.911958,-5.188723,-8.705833,-2.639307,-10.641049,4.177446,-5.480078,0
6976,file17480.mp3.wav_16k.wav_norm.wav_mono.wav_si...,107.656942,23.047497,531.187114,76.224851,148.967746,0.455764,-1.181660,0.204032,0.193788,...,-17.754799,-2.489033,-13.970686,-4.413645,-15.212324,-2.216567,-10.173335,3.313224,-6.365371,0


In [6]:
train_real = get_features(train_real_names, train_path_real)
train_real['label'] = 1
train_real

/var/folders/99/6591xgpn4sq1ypfgnrzy_7n80000gn/T/ipykernel_42598/3313768297.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return x.mean(),x.std(),x.var(),x.min(), x.max(), stats.skew(x),stats.kurtosis(x)


,0,f0_mean,f0_std,f0_var,f0_min,f0_max,f0_skew,f0_kurtosis,zcr_mean,zcr_std,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,label
0,file9328.wav_16k.wav_norm.wav_mono.wav_silence...,136.489478,4.727093,22.345412,129.683955,141.421356,-0.356493,-1.656474,0.181331,0.089871,...,-4.534492,-1.690782,-12.046480,-4.170325,-3.767523,-3.091619,-1.661529,-2.932548,-4.666971,1
1,file10017.wav_16k.wav_norm.wav_mono.wav_silenc...,90.797509,6.355125,40.387610,79.370053,106.560050,0.275807,-0.026169,0.077241,0.075381,...,3.980865,-2.343286,-8.728118,-3.408345,-9.232826,0.520565,-1.781361,-4.874956,-3.132875,1
2,file23340.wav_16k.wav_norm.wav_mono.wav_silenc...,112.195822,12.166061,148.013045,79.829839,131.950791,-1.247991,1.324854,0.179184,0.140710,...,-11.596682,-7.242671,-10.442345,-8.342333,-10.776612,-6.005401,-9.705394,-8.490805,0.352576,1
3,file32131.wav_16k.wav_norm.wav_mono.wav_silenc...,124.043074,16.197731,262.366476,103.526492,148.967746,0.184060,-1.462678,0.078156,0.031284,...,-4.495624,-0.687855,-11.527680,1.988483,0.915861,-12.134728,0.715337,0.200326,-2.796401,1
4,file32427.wav_16k.wav_norm.wav_mono.wav_silenc...,123.235478,7.022760,49.319152,108.422687,140.606829,0.142617,0.642629,0.157707,0.092378,...,-12.843907,-10.046461,-0.838059,-5.776786,-12.300733,-1.190977,-7.238159,6.845308,6.503428,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6973,file5407.wav_16k.wav_norm.wav_mono.wav_silence...,127.276709,10.326879,106.644434,110.956947,148.109755,0.260784,-0.856636,0.052750,0.062822,...,2.562416,5.660911,-1.335221,3.341729,-0.937937,-2.816634,3.509883,-0.443187,3.239182,1
6974,file28655.wav_16k.wav_norm.wav_mono.wav_silenc...,106.704239,22.497184,506.123283,74.915354,143.893358,-0.019481,-1.195028,0.131309,0.101469,...,-3.902764,8.474739,-3.548040,0.477283,0.807014,-1.982696,-0.452561,-0.672239,-1.506868,1
6975,file34583.wav_16k.wav_norm.wav_mono.wav_silenc...,105.737230,20.582710,423.647960,77.557238,148.109755,0.865426,-0.578878,0.107546,0.085380,...,1.564494,-3.931391,2.064817,1.317156,-1.618073,-1.363072,2.464074,-3.800610,2.123342,1
6976,file10951.wav_16k.wav_norm.wav_mono.wav_silenc...,115.778554,20.875081,435.769004,82.645032,149.830708,-0.355453,-0.917043,0.124705,0.091892,...,-4.022002,-12.427608,-13.461220,-7.889135,-12.129333,-6.088712,-4.669352,-7.442616,-4.684886,1


In [7]:
train = pd.concat([train_fake, train_real])
train

,0,f0_mean,f0_std,f0_var,f0_min,f0_max,f0_skew,f0_kurtosis,zcr_mean,zcr_std,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,label
0,file24569.mp3.wav_16k.wav_norm.wav_mono.wav_si...,113.146319,16.642187,276.962402,85.559503,143.064593,0.084161,-1.177386,0.164900,0.162481,...,-20.343092,-2.797815,-12.452229,-5.840678,-10.560199,0.486194,-7.003640,7.888074,-7.005135,0
1,file1886.mp3.wav_16k.wav_norm.wav_mono.wav_sil...,116.304929,20.466395,418.873322,89.605964,149.830708,0.278453,-1.560141,0.149197,0.125664,...,-10.500713,-4.870155,-14.144915,-8.112008,-5.849154,-8.864585,-3.883942,-4.553544,-5.014913,0
2,file5633.mp3.wav_16k.wav_norm.wav_mono.wav_sil...,101.062877,18.909091,357.553741,76.666417,139.796993,0.457587,-1.057336,0.101966,0.061466,...,-12.358951,2.754946,-2.687284,0.354355,-7.595921,2.471754,-6.340443,4.280274,-1.238021,0
3,file27375.wav_16k.wav_norm.wav_mono.wav_silenc...,111.522149,16.696594,278.776240,86.055144,147.256706,0.099230,-1.191811,0.183400,0.174183,...,-5.718417,-9.363832,-5.133481,-6.674107,-8.906966,-1.871459,-4.009862,8.324374,7.092777,0
4,file17442.mp3.wav_16k.wav_norm.wav_mono.wav_si...,99.681495,16.756603,280.783737,75.785828,147.256706,1.289548,1.524926,0.164946,0.137211,...,-12.736900,4.043910,-4.198050,3.157935,-5.054732,7.856312,-9.573131,5.844799,-4.269224,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6973,file5407.wav_16k.wav_norm.wav_mono.wav_silence...,127.276709,10.326879,106.644434,110.956947,148.109755,0.260784,-0.856636,0.052750,0.062822,...,2.562416,5.660911,-1.335221,3.341729,-0.937937,-2.816634,3.509883,-0.443187,3.239182,1
6974,file28655.wav_16k.wav_norm.wav_mono.wav_silenc...,106.704239,22.497184,506.123283,74.915354,143.893358,-0.019481,-1.195028,0.131309,0.101469,...,-3.902764,8.474739,-3.548040,0.477283,0.807014,-1.982696,-0.452561,-0.672239,-1.506868,1
6975,file34583.wav_16k.wav_norm.wav_mono.wav_silenc...,105.737230,20.582710,423.647960,77.557238,148.109755,0.865426,-0.578878,0.107546,0.085380,...,1.564494,-3.931391,2.064817,1.317156,-1.618073,-1.363072,2.464074,-3.800610,2.123342,1
6976,file10951.wav_16k.wav_norm.wav_mono.wav_silenc...,115.778554,20.875081,435.769004,82.645032,149.830708,-0.355453,-0.917043,0.124705,0.091892,...,-4.022002,-12.427608,-13.461220,-7.889135,-12.129333,-6.088712,-4.669352,-7.442616,-4.684886,1


In [8]:
test_fake = get_features(test_fake_names, test_path_fake)
test_fake['label'] = 0
test_fake

,0,f0_mean,f0_std,f0_var,f0_min,f0_max,f0_skew,f0_kurtosis,zcr_mean,zcr_std,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,label
0,file433.wav_16k.wav_norm.wav_mono.wav_silence....,114.657629,17.063435,291.160818,81.695773,149.830708,-0.204652,-0.218891,0.142307,0.161818,...,-0.405546,-13.376277,-3.076412,-5.982931,-11.315242,0.856092,-12.025939,0.913260,-11.271635,0
1,file2194.wav_16k.wav_norm.wav_mono.wav_silence...,130.668198,9.017135,81.308729,114.208234,148.109755,0.106843,-0.723627,0.111010,0.111368,...,1.684451,-11.411838,-4.846194,-6.104938,-9.741377,2.274526,-12.481531,2.784695,-12.102964,0
2,file1551.wav_16k.wav_norm.wav_mono.wav_silence...,122.613931,13.243914,175.401263,98.282060,148.967746,0.192298,-0.748307,0.097664,0.110241,...,2.545182,-13.576818,-3.687814,-4.145748,-12.060222,1.285453,-12.949003,1.417001,-11.506231,0
3,file1047.wav_16k.wav_norm.wav_mono.wav_silence...,119.664392,19.624439,385.118605,97.153194,149.830708,0.150561,-1.694964,0.069599,0.034497,...,-0.280570,-18.221024,-0.481088,-6.795421,-9.312627,4.091344,-14.207133,3.683449,-13.111985,0
4,file1802.wav_16k.wav_norm.wav_mono.wav_silence...,110.506493,16.689550,278.541067,87.055056,146.408570,0.428767,-0.820130,0.135107,0.141607,...,-5.446749,-12.151406,1.295001,-22.025723,-7.067898,-12.497729,-6.048635,0.552761,-6.197958,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,file296.wav_16k.wav_norm.wav_mono.wav_silence....,124.824700,16.588304,275.171827,97.715997,149.830708,0.176178,-1.402977,0.174301,0.144751,...,2.188824,-13.289555,-1.798151,-3.200665,-10.075131,2.662696,-9.651749,2.201181,-9.269334,0
540,file25.wav_16k.wav_norm.wav_mono.wav_silence.w...,100.805862,23.536922,553.986720,77.110541,148.967746,0.905629,-0.645189,0.109739,0.091916,...,-1.386792,-13.264559,-2.497263,-5.048313,-10.205741,3.289423,-12.510808,2.628861,-11.534744,0
541,file2097.wav_16k.wav_norm.wav_mono.wav_silence...,102.336547,20.653029,426.547594,71.120301,148.109755,0.635709,-0.744501,0.056555,0.031341,...,-6.710545,-16.745468,3.801952,-20.382090,-8.581886,-12.770016,-5.428394,-1.770272,-5.119710,0
542,file2227.wav_16k.wav_norm.wav_mono.wav_silence...,97.965494,20.587812,423.858004,72.363462,138.991822,0.539792,-0.994533,0.180401,0.166631,...,-6.978651,-12.092861,0.811086,-17.520975,-8.320767,-8.174894,-7.318422,0.578519,-5.694507,0


In [9]:
test_real = get_features(test_real_names, test_path_real)
test_real['label'] = 1
test_real

,0,f0_mean,f0_std,f0_var,f0_min,f0_max,f0_skew,f0_kurtosis,zcr_mean,zcr_std,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,label
0,file1342.wav_16k.wav_norm.wav_mono.wav_silence...,95.825322,12.400733,153.778191,82.169031,138.191288,1.924124,3.416352,0.132285,0.109272,...,-9.716220,-4.462096,-5.992797,-8.614376,-4.761033,-0.146127,-8.118113,2.774332,-7.364175,1
1,file1654.wav_16k.wav_norm.wav_mono.wav_silence...,87.827133,9.592132,92.008996,76.666417,123.827636,1.298637,2.118505,0.104027,0.136730,...,-8.856879,-5.666753,-2.578003,-11.782320,5.627705,-5.126607,-2.242055,2.085406,-6.525234,1
2,file1297.wav_16k.wav_norm.wav_mono.wav_silence...,123.315392,13.182852,173.787581,100.579294,145.565318,-0.242144,-1.195609,0.145585,0.131807,...,-6.665966,-4.806283,-9.407757,0.261773,-9.436312,-2.168239,-4.943616,-1.936558,-7.994697,1
3,file1781.wav_16k.wav_norm.wav_mono.wav_silence...,91.933536,17.312512,299.723084,76.666417,148.967746,2.529304,5.634170,0.127449,0.121953,...,-7.496329,-4.503344,-5.550010,-4.049525,-6.471270,5.408659,-8.878404,4.519847,-3.896475,1
4,file960.wav_16k.wav_norm.wav_mono.wav_silence....,89.199595,9.887263,97.757969,78.912915,123.827636,1.196284,1.583604,0.146167,0.125289,...,-7.753406,-5.958006,-6.685547,-5.690189,-5.948182,1.886800,-9.388988,3.660578,-8.821147,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,file296.wav_16k.wav_norm.wav_mono.wav_silence....,87.755279,6.260647,39.195705,77.557238,102.337389,0.705383,-0.306721,0.112436,0.095034,...,-9.938003,-2.820818,-3.143373,-5.545812,-3.904826,0.637030,-6.862724,0.703628,-3.113215,1
540,file25.wav_16k.wav_norm.wav_mono.wav_silence.w...,127.699253,12.044668,145.074039,94.387431,148.967746,-0.692560,0.423308,0.180377,0.136152,...,-7.028303,0.500321,-10.120844,2.981781,-10.883287,1.992162,-4.688036,0.999515,-4.312905,1
541,file1394.wav_16k.wav_norm.wav_mono.wav_silence...,120.582895,10.135408,102.726504,102.930224,143.064593,0.613238,-0.620885,0.094207,0.084008,...,-8.725149,-3.037408,-11.772900,0.752847,-10.807492,-2.045947,-2.685781,-3.337871,-5.683969,1
542,file863.wav_16k.wav_norm.wav_mono.wav_silence....,121.339347,13.423158,180.181165,104.126215,145.565318,0.490667,-1.326344,0.103306,0.087229,...,-4.031785,-9.280605,-7.707674,-0.417946,-11.308434,-0.261955,-6.187222,-1.308103,-5.724790,1


In [10]:
test = pd.concat([test_fake, test_real])
test

,0,f0_mean,f0_std,f0_var,f0_min,f0_max,f0_skew,f0_kurtosis,zcr_mean,zcr_std,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,label
0,file433.wav_16k.wav_norm.wav_mono.wav_silence....,114.657629,17.063435,291.160818,81.695773,149.830708,-0.204652,-0.218891,0.142307,0.161818,...,-0.405546,-13.376277,-3.076412,-5.982931,-11.315242,0.856092,-12.025939,0.913260,-11.271635,0
1,file2194.wav_16k.wav_norm.wav_mono.wav_silence...,130.668198,9.017135,81.308729,114.208234,148.109755,0.106843,-0.723627,0.111010,0.111368,...,1.684451,-11.411838,-4.846194,-6.104938,-9.741377,2.274526,-12.481531,2.784695,-12.102964,0
2,file1551.wav_16k.wav_norm.wav_mono.wav_silence...,122.613931,13.243914,175.401263,98.282060,148.967746,0.192298,-0.748307,0.097664,0.110241,...,2.545182,-13.576818,-3.687814,-4.145748,-12.060222,1.285453,-12.949003,1.417001,-11.506231,0
3,file1047.wav_16k.wav_norm.wav_mono.wav_silence...,119.664392,19.624439,385.118605,97.153194,149.830708,0.150561,-1.694964,0.069599,0.034497,...,-0.280570,-18.221024,-0.481088,-6.795421,-9.312627,4.091344,-14.207133,3.683449,-13.111985,0
4,file1802.wav_16k.wav_norm.wav_mono.wav_silence...,110.506493,16.689550,278.541067,87.055056,146.408570,0.428767,-0.820130,0.135107,0.141607,...,-5.446749,-12.151406,1.295001,-22.025723,-7.067898,-12.497729,-6.048635,0.552761,-6.197958,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,file296.wav_16k.wav_norm.wav_mono.wav_silence....,87.755279,6.260647,39.195705,77.557238,102.337389,0.705383,-0.306721,0.112436,0.095034,...,-9.938003,-2.820818,-3.143373,-5.545812,-3.904826,0.637030,-6.862724,0.703628,-3.113215,1
540,file25.wav_16k.wav_norm.wav_mono.wav_silence.w...,127.699253,12.044668,145.074039,94.387431,148.967746,-0.692560,0.423308,0.180377,0.136152,...,-7.028303,0.500321,-10.120844,2.981781,-10.883287,1.992162,-4.688036,0.999515,-4.312905,1
541,file1394.wav_16k.wav_norm.wav_mono.wav_silence...,120.582895,10.135408,102.726504,102.930224,143.064593,0.613238,-0.620885,0.094207,0.084008,...,-8.725149,-3.037408,-11.772900,0.752847,-10.807492,-2.045947,-2.685781,-3.337871,-5.683969,1
542,file863.wav_16k.wav_norm.wav_mono.wav_silence....,121.339347,13.423158,180.181165,104.126215,145.565318,0.490667,-1.326344,0.103306,0.087229,...,-4.031785,-9.280605,-7.707674,-0.417946,-11.308434,-0.261955,-6.187222,-1.308103,-5.724790,1


In [11]:
valid_fake = get_features(valid_fake_names, validation_path_fake)
valid_fake['label'] = 0
valid_fake

,0,f0_mean,f0_std,f0_var,f0_min,f0_max,f0_skew,f0_kurtosis,zcr_mean,zcr_std,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,label
0,file788.mp3.wav_16k.wav_norm.wav_mono.wav_sile...,136.813780,8.729620,76.206267,101.747969,148.109755,-1.564041,3.818840,0.132270,0.120635,...,3.385038,-17.438786,-10.179822,5.373208,-3.794394,19.132391,10.359378,12.972460,5.344854,0
1,file28433.wav_16k.wav_norm.wav_mono.wav_silenc...,109.261443,13.717945,188.182022,91.700404,145.565318,0.977922,-0.315951,0.173355,0.130249,...,-16.228827,-6.682185,-2.542838,-6.784031,-4.679899,-2.703435,-6.633286,1.896547,-7.040866,0
2,file19302.mp3.wav_16k.wav_norm.wav_mono.wav_si...,102.249812,14.694031,215.914548,78.912915,147.256706,0.732109,0.474213,0.107678,0.089856,...,-1.631543,-1.669028,-9.276979,0.401367,-3.886419,-4.069921,-2.708086,1.215653,-5.558532,0
3,file30232.mp3.wav_16k.wav_norm.wav_mono.wav_si...,122.194141,14.415081,207.794567,91.172249,148.109755,-0.490192,-0.462632,0.075637,0.033315,...,0.517548,-17.974178,-2.031919,-6.728065,-12.158307,1.673301,-11.036796,-3.131088,-3.960863,0
4,file9463.mp3.wav_16k.wav_norm.wav_mono.wav_sil...,110.578947,29.832291,889.965566,74.054878,148.967746,0.115775,-1.785342,0.097222,0.078869,...,-3.590478,-17.051195,0.108770,-11.435762,-10.675420,1.270589,-14.656271,-2.911393,-5.560000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,file21181.mp3.wav_16k.wav_norm.wav_mono.wav_si...,108.776333,23.570935,555.588976,77.557238,142.240602,0.087376,-1.634465,0.119993,0.088939,...,-5.058923,-13.315575,0.241012,-9.803412,-10.873135,0.198975,-8.545796,-3.737772,-7.436629,0
1409,file25300.mp3.wav_16k.wav_norm.wav_mono.wav_si...,128.391681,13.832099,191.326970,97.715997,148.967746,-0.433861,-0.759456,0.079683,0.046303,...,-1.690111,-13.018595,-8.074149,-0.201076,-0.522111,15.634154,6.906502,11.327919,4.571111,0
1410,file4030.wav_16k.wav_norm.wav_mono.wav_silence...,114.931581,16.500375,272.262383,84.089642,149.830708,0.631320,-0.408643,0.152181,0.121332,...,-19.126173,-5.585559,-4.001717,-9.504446,-4.028611,-3.395478,-5.930574,-0.065886,-6.740932,0
1411,file868.mp3.wav_16k.wav_norm.wav_mono.wav_sile...,108.224041,11.394335,129.830862,87.055056,146.408570,1.201796,1.401562,0.143338,0.108396,...,-14.944983,-0.046967,-17.327759,-4.730996,-12.841231,-1.053484,-11.739232,2.241216,-10.125710,0


In [12]:
valid_real = get_features(valid_real_names, validation_path_real)
valid_real['label'] = 1
valid_real

/var/folders/99/6591xgpn4sq1ypfgnrzy_7n80000gn/T/ipykernel_42598/3313768297.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return x.mean(),x.std(),x.var(),x.min(), x.max(), stats.skew(x),stats.kurtosis(x)


,0,f0_mean,f0_std,f0_var,f0_min,f0_max,f0_skew,f0_kurtosis,zcr_mean,zcr_std,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,label
0,file29361.wav_16k.wav_norm.wav_mono.wav_silenc...,117.759562,14.416919,207.847547,98.282060,141.421356,0.410929,-1.253992,0.193398,0.174873,...,-7.647556,-5.092465,-4.550917,-2.290437,-5.071510,-0.700512,-7.836610,2.244458,-3.857556,1
1,file30076.wav_16k.wav_norm.wav_mono.wav_silenc...,98.797523,8.517785,72.552659,87.055056,116.877725,0.771318,-0.716293,0.173069,0.164678,...,-17.317301,-6.116851,-9.113156,-6.791862,-10.089137,-1.487490,-7.531910,-0.092652,-7.301184,1
2,file22412.wav_16k.wav_norm.wav_mono.wav_silenc...,93.996606,6.244166,38.989607,84.089642,112.896440,0.922417,0.935484,0.068034,0.066129,...,-15.856291,-9.381164,-9.060288,-11.346145,-7.595401,-5.472383,-10.289623,-2.214857,-9.279985,1
3,file30833.wav_16k.wav_norm.wav_mono.wav_silenc...,129.614321,22.553288,508.650816,96.037295,149.830708,-0.699034,-1.467646,0.221160,0.133443,...,-4.338401,-1.363620,-6.289300,1.985072,-5.657739,-0.251511,-5.058565,-1.221819,-1.642941,1
4,file32742.wav_16k.wav_norm.wav_mono.wav_silenc...,61.199499,3.426118,11.738287,59.804806,79.370053,4.556347,21.043061,0.088208,0.085119,...,-0.565947,1.564183,-2.605452,0.096519,2.549134,3.971712,3.586417,0.727740,1.191762,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,file34225.wav_16k.wav_norm.wav_mono.wav_silenc...,129.530716,9.401304,88.384523,114.869835,148.967746,0.511307,-0.556439,0.105329,0.113641,...,-1.282192,2.131546,-0.363361,-1.330848,-3.010127,0.636304,-0.444742,-3.092845,-1.559147,1
1409,file33509.wav_16k.wav_norm.wav_mono.wav_silenc...,120.503314,8.434247,71.136526,109.682498,141.421356,1.073104,0.792451,0.034226,0.036938,...,2.399934,3.241878,-0.915863,1.024190,-6.576999,5.953642,-1.329071,1.359565,1.649894,1
1410,file32877.wav_16k.wav_norm.wav_mono.wav_silenc...,129.656921,8.249900,68.060855,113.550443,146.408570,-0.149195,-0.953504,0.071196,0.050251,...,2.281144,-1.244002,-1.738579,2.075552,-5.506531,-2.162258,0.656441,-5.794055,-2.812670,1
1411,file29897.wav_16k.wav_norm.wav_mono.wav_silenc...,108.804999,15.692121,246.242659,86.055144,149.830708,0.313629,-0.465293,0.168015,0.139705,...,-9.577928,-9.639128,-8.662293,-7.259605,-9.880514,-0.176027,-6.296833,-0.170614,-4.396715,1


In [13]:
validation = pd.concat([valid_fake, valid_real])
validation

,0,f0_mean,f0_std,f0_var,f0_min,f0_max,f0_skew,f0_kurtosis,zcr_mean,zcr_std,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,label
0,file788.mp3.wav_16k.wav_norm.wav_mono.wav_sile...,136.813780,8.729620,76.206267,101.747969,148.109755,-1.564041,3.818840,0.132270,0.120635,...,3.385038,-17.438786,-10.179822,5.373208,-3.794394,19.132391,10.359378,12.972460,5.344854,0
1,file28433.wav_16k.wav_norm.wav_mono.wav_silenc...,109.261443,13.717945,188.182022,91.700404,145.565318,0.977922,-0.315951,0.173355,0.130249,...,-16.228827,-6.682185,-2.542838,-6.784031,-4.679899,-2.703435,-6.633286,1.896547,-7.040866,0
2,file19302.mp3.wav_16k.wav_norm.wav_mono.wav_si...,102.249812,14.694031,215.914548,78.912915,147.256706,0.732109,0.474213,0.107678,0.089856,...,-1.631543,-1.669028,-9.276979,0.401367,-3.886419,-4.069921,-2.708086,1.215653,-5.558532,0
3,file30232.mp3.wav_16k.wav_norm.wav_mono.wav_si...,122.194141,14.415081,207.794567,91.172249,148.109755,-0.490192,-0.462632,0.075637,0.033315,...,0.517548,-17.974178,-2.031919,-6.728065,-12.158307,1.673301,-11.036796,-3.131088,-3.960863,0
4,file9463.mp3.wav_16k.wav_norm.wav_mono.wav_sil...,110.578947,29.832291,889.965566,74.054878,148.967746,0.115775,-1.785342,0.097222,0.078869,...,-3.590478,-17.051195,0.108770,-11.435762,-10.675420,1.270589,-14.656271,-2.911393,-5.560000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,file34225.wav_16k.wav_norm.wav_mono.wav_silenc...,129.530716,9.401304,88.384523,114.869835,148.967746,0.511307,-0.556439,0.105329,0.113641,...,-1.282192,2.131546,-0.363361,-1.330848,-3.010127,0.636304,-0.444742,-3.092845,-1.559147,1
1409,file33509.wav_16k.wav_norm.wav_mono.wav_silenc...,120.503314,8.434247,71.136526,109.682498,141.421356,1.073104,0.792451,0.034226,0.036938,...,2.399934,3.241878,-0.915863,1.024190,-6.576999,5.953642,-1.329071,1.359565,1.649894,1
1410,file32877.wav_16k.wav_norm.wav_mono.wav_silenc...,129.656921,8.249900,68.060855,113.550443,146.408570,-0.149195,-0.953504,0.071196,0.050251,...,2.281144,-1.244002,-1.738579,2.075552,-5.506531,-2.162258,0.656441,-5.794055,-2.812670,1
1411,file29897.wav_16k.wav_norm.wav_mono.wav_silenc...,108.804999,15.692121,246.242659,86.055144,149.830708,0.313629,-0.465293,0.168015,0.139705,...,-9.577928,-9.639128,-8.662293,-7.259605,-9.880514,-0.176027,-6.296833,-0.170614,-4.396715,1


In [14]:
y_test = test['label']
x_test = test.drop(columns=['label'])

In [15]:
y_train = train['label']
x_train = train.drop(columns=['label'])

In [16]:
y_valid = validation['label']
x_valid = validation.drop(columns = ['label'])

In [18]:
train.to_csv('finalproject_train.csv', index=False)
test.to_csv('finalproject_test.csv', index=False)
validation.to_csv('finalproject_validation.csv', index=False)

---